
# NCars Event-GNN Training (SAGE/GCN) — with AEGNN Async Eval

- Loads NCars sequences from `{train, validation, test}`
- Builds k-NN graphs in (x, y, t)
- Trains a **SAGE** or **GCN** encoder + classifier head
- Optionally wraps the **encoder only** with `AEGNNAsyncWrapper` for asynchronous evaluation
- Lets us limit the number of sequences per split for quick tests




In [1]:

# data folder containing 'training/', 'validation/', 'test/'
DATA = r"C:\Users\hanne\Documents\Hannes\Uni\Maastricht\Project\GNNBenchmark\src\Models\AEGNN\data\ncars"

# If your events file has a specific name (else the dataset will auto-pick the first *.txt != is_car.txt)
EVENTS_NAME = None  # should be "events.txt"

# Sensor width,height (string "W,H") or None to normalize per-sequence
SENSOR_WH = "304,240"  # or None

# Graph building
K = 8
MAX_EVENTS = 1000  # reduce (e.g., 1000) for faster first run

# Model
MODEL = "sage"     # "sage" or "gcn"
HIDDEN = 64
DROPOUT = 0.2

# Training
EPOCHS = 30
BATCH_SIZE = 32
LR = 1e-3
WD = 0.0

# Use only first N (20) sequences per split (0 = all), for quick testing
LIMIT_PER_SPLIT = 20

# Async encoder wrapping at evaluation
ASYNC_EVAL = True


In [2]:

import sys, os, pathlib
project_root_candidates = [
    pathlib.Path.cwd(),                                  # current directory
    pathlib.Path.cwd().parents[0],                       # 1 level up
    pathlib.Path.cwd().parents[1],                       # 2 levels up
]
for pr in project_root_candidates:
    if str(pr) not in sys.path:
        sys.path.append(str(pr))

from pathlib import Path
import torch
import torch.nn as nn
from torch.optim import Adam
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# moduls needed
from ProcessNCars import NCarsEventsGraphDataset
from GCNEncoder import GCNEncoder
from SAGEEncoder import SAGEEncoder
from FullModel_GCN_SAGE import FullModel, ClassifierHead
from train_ncars_from_events import train_one_epoch, evaluate, take_first


#

# Data Loading


In [3]:
# dataset setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sensor_wh = None
if SENSOR_WH:
    w, h = [int(s.strip()) for s in SENSOR_WH.split(",")]
    sensor_wh = (w, h)

root = Path(DATA)
train_dir = root / "training"#change to your data path
val_dir   = root / "validation"
test_dir  = root / "test"
for d in [train_dir, val_dir, test_dir]:
    assert d.exists(), f"Missing split directory: {d}"

precompute_gcn = (MODEL == "gcn")

train_ds = NCarsEventsGraphDataset(str(train_dir), EVENTS_NAME, K, MAX_EVENTS,
                                   sensor_wh, precompute_gcn, cache=True)
val_ds   = NCarsEventsGraphDataset(str(val_dir),   EVENTS_NAME, K, MAX_EVENTS,
                                   sensor_wh, precompute_gcn, cache=True)
test_ds  = NCarsEventsGraphDataset(str(test_dir),  EVENTS_NAME, K, MAX_EVENTS,
                                   sensor_wh, precompute_gcn, cache=True)

# Limit for quick runs and testing
if LIMIT_PER_SPLIT and LIMIT_PER_SPLIT > 0:
    train_ds = take_first(train_ds, LIMIT_PER_SPLIT)
    val_ds   = take_first(val_ds, LIMIT_PER_SPLIT)
    # Uncomment to also limit test set:
    test_ds  = take_first(test_ds, LIMIT_PER_SPLIT)

print("Datasets ready.",
      "\n  train:", len(train_ds),
      "\n  val:  ", len(val_ds),
      "\n  test: ", len(test_ds))


Datasets ready. 
  train: 20 
  val:   20 
  test:  50


# Model, Loaders, Optimizer


In [4]:
# model
sample: Data = train_ds[0]
in_ch = sample.x.size(-1)
num_classes = int(max(sample.y.max().item(), 1) + 1)

if MODEL == "sage":
    encoder = SAGEEncoder(in_ch, hid=HIDDEN)
else:
    encoder = GCNEncoder(in_ch, hid=HIDDEN)

head = ClassifierHead(hid=HIDDEN, num_classes=num_classes, dropout=DROPOUT, bias=True)
model = FullModel(encoder, head).to(device)

# loaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False)

# Optimizer & loss
opt = Adam(model.parameters(), lr=LR, weight_decay=WD)
crit = nn.CrossEntropyLoss()

print(model)


FullModel(
  (encoder): SAGEEncoder(
    (c1): SAGEConv(4, 64, aggr=mean)
    (c2): SAGEConv(64, 64, aggr=mean)
  )
  (head): ClassifierHead(
    (fc): Linear(in_features=64, out_features=2, bias=True)
  )
)


C:\Users\hanne\Documents\Hannes\Uni\Maastricht\Project\GNNBenchmark\src\Models\AEGNN\ProcessNCars.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cach

# Training Loop

In [5]:

#training loop
best_val, best_state = 0.0, None
for epoch in range(1, EPOCHS + 1):
    tr_loss, tr_acc = train_one_epoch(model, train_loader, device, opt, crit)
    va_loss, va_acc = evaluate(model, val_loader, device, crit)
    if va_acc > best_val:
        best_val = va_acc
        best_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}
    print(f"Epoch {epoch:03d} | train {tr_loss:.4f}/{tr_acc:.3f} | val {va_loss:.4f}/{va_acc:.3f}")

if best_state:
    model.load_state_dict(best_state)


Epoch 001 | train 0.6998/0.500 | val 0.7109/0.450
Epoch 002 | train 0.6981/0.450 | val 0.7058/0.450
Epoch 003 | train 0.6862/0.450 | val 0.7015/0.450
Epoch 004 | train 0.6973/0.550 | val 0.6987/0.350
Epoch 005 | train 0.7038/0.350 | val 0.6966/0.450
Epoch 006 | train 0.6887/0.550 | val 0.6953/0.500
Epoch 007 | train 0.6951/0.450 | val 0.6947/0.550
Epoch 008 | train 0.7122/0.400 | val 0.6944/0.550
Epoch 009 | train 0.7096/0.350 | val 0.6950/0.550
Epoch 010 | train 0.6886/0.550 | val 0.6958/0.550
Epoch 011 | train 0.6864/0.500 | val 0.6968/0.550
Epoch 012 | train 0.6804/0.650 | val 0.6980/0.500
Epoch 013 | train 0.6700/0.700 | val 0.6995/0.450
Epoch 014 | train 0.6977/0.300 | val 0.7012/0.400
Epoch 015 | train 0.6883/0.650 | val 0.7029/0.300
Epoch 016 | train 0.6833/0.650 | val 0.7044/0.200
Epoch 017 | train 0.6809/0.600 | val 0.7060/0.350
Epoch 018 | train 0.7110/0.450 | val 0.7072/0.450
Epoch 019 | train 0.6840/0.600 | val 0.7077/0.450
Epoch 020 | train 0.6895/0.550 | val 0.7085/0.400


# Asynchronous Evaluation

In [6]:


# async for evaluation, only part taken from AEGNN
if ASYNC_EVAL:
    try:
        from AEGNNwrapper import AEGNNAsyncWrapper
        enc_async = AEGNNAsyncWrapper(model.encoder)
        print("[AEGNN] is_async=", getattr(enc_async, "is_async", False),
              "| why_not=", getattr(enc_async, "why_not_async", None))
        model.encoder = enc_async.to(device)
    except Exception as e:
        print("[AEGNN] Wrapper import failed; continuing without async. Reason:", repr(e))


[AEGNN] is_async= True | why_not= None


In [7]:
from tqdm import tqdm  # pip install tqdm (optional); else replace with a simple loop
bad = []
for i in range(len(test_ds)):
    try:
        _ = test_ds[i]
    except Exception as e:
        bad.append((i, str(test_ds.seq_dirs[i]), repr(e)))
        print("BAD:", i, test_ds.seq_dirs[i], e)

print("bad count:", len(bad))
if bad:
    print("examples:", bad[:5])


bad count: 0


# Testing


In [ ]:
# test
te_loss, te_acc = evaluate(model, test_loader, device, crit)
print(f"Test  | loss {te_loss:.4f} acc {te_acc:.3f}")